# Introduction of new Python Package/Tool
---
Installation notes and any related comments...

In [ ]:
!pip install pydantic-settings

In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install pandas-profiling[notebook,html] --user

In [1]:
# All import statements 
# Packages Explored in Module
import pandas as pd 
import numpy
import matplotlib

# New Tool for This Project
import ydata_profiling as pp

# Individually written tools from module files

# The Evolution of Musical Attributes and Their Impact on the Popularity of Songs in the Digital Era

### Problem Statement:
In the age of digital music, the success of a song is often gauged by its popularity on streaming platforms, which can be influenced by various musical attributes. Understanding how different attributes correlate with popularity can provide insights into listener preferences and inform decisions in the music production and marketing processes.

### Objective:

To analyze the relationship between various song attributes such as genre, tempo, and explicit content and their popularity scores.
To identify trends in the evolution of musical attributes over time and how they relate to shifts in listener engagement and popularity.
To determine if there are specific patterns or characteristics that are common among highly popular songs.

### Hypotheses:

Songs with certain attributes, like a higher tempo or inclusion of explicit content, are more likely to achieve high popularity scores.
Changes in listener preferences over time will reflect in the prominence of different musical attributes within the most popular songs.
There may be a pattern of convergence or divergence in song attributes within top-charting hits compared to songs with lower popularity scores.

### Research Questions:

What song attributes most strongly correlate with high popularity scores within the dataset?
How have the common attributes of popular songs changed over the past decade?
Can we predict the potential popularity of a song based on its attributes without considering its audio features?

### Significance:
The project's findings can offer valuable insights for artists, producers, and record labels into the types of song attributes that resonate with listeners in the current music landscape. Additionally, these insights could be used to develop more targeted music recommendation systems that align with evolving listener preferences.

# 2. Collect Data
---

dataset is MusicOSet

In [2]:
import os

# Get the current working directory
cwd = os.getcwd()

# Construct the path to the "Data" folder
data_folder = os.path.join(cwd, "Data")

# Initialize an empty list to store the data frames
data_frames = []

# Loop through all files in the "Data" folder
for file in os.listdir(data_folder):
    # Check if the file is a CSV
    if file.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(data_folder, file)
        
        try:
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Append the DataFrame to the list
            data_frames.append(df)
        
        except pd.errors.ParserError as e:
            print(f"Parser error occurred while reading {file}: {e}")
        
        except Exception as e:
            print(f"An error occurred while reading {file}: {e}")

# Concatenate all the data frames into a single DataFrame
if data_frames:
    data = pd.concat(data_frames, ignore_index=True)
    print("Data frames concatenated successfully.")
else:
    print("No valid data frames found.")


Parser error occurred while reading albums.csv: Error tokenizing data. C error: Expected 1 fields in line 39, saw 3

Parser error occurred while reading artists.csv: Error tokenizing data. C error: Expected 3 fields in line 3, saw 6

Parser error occurred while reading songs.csv: Error tokenizing data. C error: Expected 4 fields in line 113, saw 5

Data frames concatenated successfully.


# 3. Exploratory Data Analysis
---
Comments...

In [3]:
# Exploration code
print(data.head())

  song_id\tduration_ms\tkey\tmode\ttime_signature\tacousticness\tdanceability\tenergy\tinstrumentalness\tliveness\tloudness\tspeechiness\tvalence\ttempo  \
0  3e9HZxeyfWwjeyPAMmWSSQ\t207320\t1\t1\t4\t0.229...                                                                                                       
1  5p7ujcrUXASCNwRaWNHR1C\t201661\t6\t1\t4\t0.297...                                                                                                       
2  2xLMifQCjDGFmkHkpNLD9h\t312820\t8\t1\t4\t0.005...                                                                                                       
3  3KkXRkHbMCARz0aVfEt68P\t158040\t2\t1\t4\t0.556...                                                                                                       
4  1rqqCSm0Qe4I9rUvWncaom\t190947\t5\t1\t4\t0.193...                                                                                                       

  album_id\trank_score\tpeak_position\tweek_count\tdate  \
0   

In [ ]:
# Figures

Descriptions and discoveries...

In [ ]:
# Exploration code

In [ ]:
# Figures

Descriptions and discoveries...

In [ ]:
# Exploration code (potential integration of module code)

In [ ]:
# Figures (potential integration of module code)

Descriptions and discoveries...

# 4. Data Pre-Processing
---
Explanations...

In [ ]:
# Data manipulation 1 (potential integration of module code)

In [ ]:
# Data manipulation 2 (potential integration of module code)

Related discussion...

# 5. In-Depth Analysis
---
Explanations...

In [ ]:
# Implementations (potential integration of module code)

Descriptions and discoveries...

In [ ]:
# Visualisations (potential integration of module code)

Descriptions and discoveries...

In [ ]:
# Evaluations (potential integration of module code)

Descriptions and discoveries...

In [ ]:
# Visualisations (potential integration of module code)

Descriptions and discoveries...

# 6. Communicate Results
---
Overall conclusions...

# References
---
...

# Group Reflection

---

Comments...

---
*Optional Unequal Contribution Table*
Member | Contribution [% effort]
-|-
Jamie Soan | 100%
Angus Day | 100%
William Heap | 100%
kj | 100%

# Jamie Reflection
---
Comments...

# Angus Reflection
---
Comments...

# William Reflection
---
Comments...

# KJ Reflection
---
Comments...

# JKL Reflection
---
Comments...